In [1]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from torch.utils.data import TensorDataset
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,confusion_matrix, f1_score
from utils.function import *
from models.daae_final import DAE
from models.activity_recognition import *
from models.dis_z import DIS_Z
import copy
#from tqdm import tqdm

In [2]:
class Args:
    def __init__(self):
        self.root = '../data'
        self.batchSize = 64
        self.maxEpochs = 200
        self.nz = 400
        self.lr = 1e-4
        self.fSize = 64
        # self.outDir = 'data/experiments/DAAE_FINAL'
        self.commit = 'eval'
        self.alpha = 1.0
        self.M = 5
        self.loss = 'MSE' #'BCE'
        self.loadDAE = False
        # self.loadSVM = False    
        self.load_DAE_from = None
        self.evalMode = False
        self.comment = ''
        self.momentum = 0.1
        self.c = 0.01
        # self.svmLR = 1e-4
        # self.Ntest = 100
        self.multimodalZ = False
        # self.window_len = 512
        # self.stride_len = 20
        # self.act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
        self.gpuNo = 2
        self.imSize = 64
        # self.sigma = [60, 80]
        # self.sigma=0.25
        self.dataset = 'OPPO' # OPPO or PAMAP2
        self.corr = 'Gaussian'
        self.sigma = [0.1, 3, 8]
        self.random_seed = 0
        self.train_split = 0.8

        #self.corr= 'ZeroMask' # options: Gaussian, ZeroMask, ConsecutiveZeros
        # if self.corr == 'ConsecutiveZeros':
        #     self.sigma = [40, 80] # [lambda_corr, lambda_norm]

        # path for corruption"Gaussian noise for sigma=0.25"
        #self.dae_model_loc = "data/experiments/DAAE1000/Ex_1"

        # path for corruption "consecutive interval"sigma [60,80]
        # self.dae_model_loc = "data/experiments/DAAE_EVAL_Noise/Ex_3" 
        self.dae_model_loc = "data/experiments/DAAE_FINAL/Ex_7" 

        #path for corruption"zeromask", sigma =0.35
        #self.dae_model_loc = "data/experiments/DAAE1000/Ex_16"

        # self.ar_model_loc = "data/experiments/DAAE1000/Ex_136/ar_params"


In [3]:
def build_dis(args, dae, multimodalZ):
    if not multimodalZ:
        print('\n ** USING NORMAL PRIOR **')
        prior = dae.norm_prior
        NZ = args.nz
    else:
        print('\n ** USING MULTIMODAL PRIOR **')
        prior = dae.multi_prior
        NZ = 2
    dis = DIS_Z(nz=NZ, prior=prior)

    return dis, NZ



def analysis(args, dae, testDataset, X, num_classes):
    # Prepare testdata set, drop the last incomplete batch
    tail = len(testDataset) % args.batchSize
    test_x = torch.zeros(len(testDataset)-tail, X.shape[1], X.shape[2], X.shape[3])
    test_labels = torch.zeros(len(testDataset)-tail, num_classes)
    for test_id in range(len(testDataset)-tail):
        test_labels[test_id] = testDataset[test_id][1]
        test_x[test_id] = testDataset[test_id][0]
    #test_x = test_x.permute(0,2,1)
    print(test_x.shape)

    # Corrupt dataset
    corr_test_x = dae.corrupt(test_x)
    
    mean_fill_test = interpolation_meanfilling(corr_test_x)
    linear_interp_test = linear_interpolation(corr_test_x)
    print(mean_fill_test.shape)
    print(linear_interp_test.shape)
# recon_test -> synthesize the entire dataset
    recon_test = dae.decode(dae.encode(corr_test_x))

# reconstruct filling testset (fill missing values only, not valid for noisy data)
    recon_fill_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
    np.copyto(recon_fill_test, recon_test.detach().numpy(), where = recon_fill_test==0)
    recon_fill_test = torch.from_numpy(recon_fill_test)

    # Reconstruct testset
    # corr_test_dataset = TensorDataset(outputs.permute(0, 2, 1))
    #recon_test = torch.from_numpy(recon_test)#.permute(1, 2, 0)

    raw_test_dataset = TensorDataset(test_x, test_labels)
    corr_test_dataset = TensorDataset(corr_test_x, test_labels)
    recon_test_dataset = TensorDataset(recon_test, test_labels)
    recon_fill_test_dataset = TensorDataset(recon_fill_test, test_labels)
    mean_fill_test_dataset = TensorDataset(mean_fill_test, test_labels)
    linear_interp_test_dataset = TensorDataset(linear_interp_test, test_labels)

    return test_x, corr_test_x, recon_test, recon_fill_test, mean_fill_test, linear_interp_test, raw_test_dataset, corr_test_dataset, recon_test_dataset, recon_fill_test_dataset, mean_fill_test_dataset,linear_interp_test_dataset
    #return corr_test_x, test_x, recon_test, raw_test_dataset, corr_test_dataset, recon_test_dataset

In [4]:
def interpolation_meanfilling(corr_test_x):
    mean_fill_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
    for i in range(mean_fill_test.shape[0]):
        for j in range(mean_fill_test.shape[2]):
            if np.count_nonzero(mean_fill_test[i,:,j,:]) == 0:  
                ch_mean = 0
            else:
             #ch_mean = np.sum(mean_fill_test[i][0][j]) / np.count_nonzero(mean_fill_test[i][0][j])
                ch_mean = np.sum(mean_fill_test[i,:,j,:]) / np.count_nonzero(mean_fill_test[i,:,j,:])
                mean_fill_test[i,:,j,:][mean_fill_test[i,:,j,:] == 0] = ch_mean
    mean_fill_test = torch.from_numpy(mean_fill_test)
    return mean_fill_test

def linear_interpolation(corr_test_x):
    linear_interp_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
    # corr_text_x shape is 18944, 1, 27, 171
    # expected shape 18944, 171, 27
    # linear_interp_test = linear_interp_test.reshape(-1, 171,27)
    linear_interp_test = linear_interp_test.reshape(-1, linear_interp_test.shape[3],linear_interp_test.shape[2])
    for i in range(linear_interp_test.shape[0]):
        for j in range(linear_interp_test.shape[2]):
            if np.count_nonzero(linear_interp_test[i,:,j]) == 0: # when all data points in this channel are missing
                linear_interp_test[i, :, j] = 0.0
            else:
                idxs = np.arange(linear_interp_test.shape[1]) # indexes of all the samples
                zero_filter = linear_interp_test[i,:,j] == 0 # index filter for zero values
                zero_idxs = idxs[zero_filter] # indexes for zero values
                non_zero_idxs = idxs[~zero_filter] # xp, indexes for non-zero values
                non_zero_vals = linear_interp_test[i, ~zero_filter,j] # fp, non-zero values
                interp_vals = np.interp(zero_idxs, non_zero_idxs, non_zero_vals) # interpolated values
                linear_interp_test[i,zero_idxs,j] = interp_vals # fill interpolated values to the corrupted signal
    linear_interp_test = torch.from_numpy(linear_interp_test)
    linear_interp_test = torch.reshape(linear_interp_test, (-1, 1, linear_interp_test.shape[2], linear_interp_test.shape[1]))
    return linear_interp_test


#def evaluate_rmse(corr_test_x, recon_test, test_x)
def evaluate_rmse(corr_test_x, recon_test, recon_fill_test,mean_fill_test,linear_interp_test,test_x):
    corr_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().detach().numpy(), corr_test_x.reshape(corr_test_x.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Corr RMSE:\n' + str(corr_rms))

    recon_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().detach().numpy(), recon_test.reshape(recon_test.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Recon RMSE:\n' + str(recon_rms))

    recon_fill_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().detach().numpy(), recon_fill_test.reshape(recon_fill_test.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Recon fill RMSE:\n' + str(recon_fill_rms))

    mean_fill_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().detach().numpy(), mean_fill_test.reshape(mean_fill_test.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Mean Fill RMSE:\n' + str(mean_fill_rms))
    
    linear_interp_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1), linear_interp_test.reshape(linear_interp_test.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Linear Interpolation RMSE:\n' + str(linear_interp_rms))
    return 
#def plot(test_x, corr_test_x, recon_test)

def plot(test_x, corr_test_x, recon_test, recon_fill_test,mean_fill_test, linear_interp_test):
    plt.imshow(test_x[0][0].detach())
    plt.title('Raw Data')
   # plt.subplot(4,1,1)
    plt.savefig(join(args.dae_model_loc, 'raw.png'))

    plt.imshow(corr_test_x[0][0].detach())
    plt.title('Corrupted')
   # plt.subplot(4,1,2)
    plt.savefig(join(args.dae_model_loc, 'corr.png'))

    plt.imshow(recon_test[0][0].detach())
    plt.title('Reconstructed')
   # plt.subplot(4,1,3)
    plt.savefig(join(args.dae_model_loc, 'reconstructed.png'))

    plt.imshow(recon_fill_test[0][0].detach())
    plt.title('Reconstructed fill')
   # plt.subplot(4,1,3)
    plt.savefig(join(args.dae_model_loc, 'rec_fill.png'))

    plt.imshow(mean_fill_test[0][0].detach())
    plt.title('Mean Fill')
   # plt.subplot(4,1,4)
    plt.savefig(join(args.dae_model_loc, 'mean.png'))
    
    plt.imshow(linear_interp_test[0][0].detach())
    plt.title('Linear Interp')
   # plt.subplot(4,1,4)
    plt.savefig(join(args.dae_model_loc, 'linear.png'))
    
    return





def test_dae(args, path, trainLoader, testLoader):
    pass


def test_activity_recognition(args, path, trainLoader, testLoader):
    pass


def calculate_combined_accuracy(args, test_loader, sigma):
    device = torch.device(args.gpuNo if torch.cuda.is_available() else "cpu")
    
    #ar = ActivityRecognitionCNN(len(args.act_list))
    ar = get_eval_model(n_sensor_channels=args.n_sensor_channels, len_seq=args.len_seq, num_classes=args.num_classes, model_path=args.ar_model_loc) #n_sensor_channels, len_seq, num_classes, model_path
    #ar.load_state_dict(torch.load(args.ar_model_loc))
    
    ar.to(device)
    correct = 0
    total = 0
    total_true = []
    total_pred = []
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
#         for data in testLoader:
#             # images, labels = prep_data(data, useCUDA=dae.useCUDA)
#             images, labels = data
#             images = images.to(device)
#             labels = labels.to(device)
            
#             # calculate outputs by running images through the network
#             outputs = ar(images)
#             # the class with the highest energy is what we choose as prediction
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
#             #accuracy = 100 * correct // total
#     print(f'Test Accuracy: {100 * correct / total} %')
#     print('\n')
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)  
            # print(images.shape)
            
            # val_h = tuple([each.data for each in val_h])

            outputs = ar(images)

            # _, predicted = torch.max(outputs.data, 1)
            predicted = torch.argmax(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
            
            total_pred = total_pred + predicted.cpu().numpy().tolist()
            total_true = total_true + (torch.argmax(labels, dim=1).cpu().numpy().tolist())
            
    print(f'Test Accuracy: {100.0 * correct / total} %')
    
    # print(" | ".join(args.act_labels_txt))
    conf_mat = confusion_matrix(y_true = total_true, y_pred = total_pred)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print(np.array(conf_mat).round(3) * 100)  
    f1 = f1_score(y_true = total_true, y_pred = total_pred, average='weighted')
    print('F1 score:', f1)
    print('')

    return 


In [5]:
if __name__ == "__main__":
    args = Args()
    if args.multimodalZ:
        args.nz = 2
    
    if args.dataset == 'PAMAP2':
        args.ar_model_loc = "data/pamap2_ConvAttn.pt"
        args.num_classes = 12
        args.n_sensor_channels = 27
        args.len_seq = 171
        args.act_labels_txt = []
        
    elif args.dataset == 'OPPO':
        args.ar_model_loc = "data/opportunity_ConvAttn.pt"
        args.num_classes = 5
        args.n_sensor_channels = 113
        args.len_seq = 24
        args.act_labels_txt = []

        
    # random.seed(args.random_seed)
    # np.random.seed(args.random_seed)
    # torch.manual_seed(args.random_seed)

    # if args.dataset == 'PAMAP2':
    #     X, labels = prepare_data(args)
    #     trainDataset, testDataset, trainLoader, testLoader = prepare_dataloaders(args, X, labels)
    
    if args.dataset == 'PAMAP2':
        X_train, X_test, y_train, y_test = prepare_data_PAMAP2(args) # x.shape = (94895, 1, 27, 171)
        # dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(labels))
        # Train/Test dataset split
        # train_size = int(args.train_split * len(dataset))
        # test_size = len(dataset) - train_size
        # trainDataset, testDataset = torch.utils.data.random_split(dataset, [train_size, test_size])        
    elif args.dataset == 'OPPO':
        X_train, X_test, y_train, y_test = prepare_data_OPPO()
    testDataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
        # trainLoader, testLoader = prepare_dataloaders_OPPO(args, X_train, X_test, y_train, y_test)        
    
    dae = DAE(nz=args.nz, corr=args.corr, sigma=args.sigma, dataset=args.dataset, fSize=args.fSize, multimodalZ=args.multimodalZ)
    dae.load_params(args.dae_model_loc)

    # test_dae(args, dae_path, trainLoader, testLoader)
    # test_activity_recognition(args, ar_path, trainLoader, testLoader)

    #acc = calculate_combined_accuracy(args, testLoader, sigma=0.5)
    test_x, corr_test_x, recon_test, recon_fill_test, mean_fill_test, linear_interp_test, raw_test_dataset, corr_test_dataset, recon_test_dataset, recon_fill_test_dataset, mean_fill_test_dataset, linear_interp_test_dataset = analysis(args, dae, testDataset, X_test, args.num_classes)
    #corr_test_x, test_x, recon_test, raw_test_dataset, corr_test_dataset, recon_test_dataset = analysis(dae, testDataset, labels)

    

    print("Raw testset:")
    raw_test_loader = torch.utils.data.DataLoader(raw_test_dataset,
    batch_size= args.batchSize, shuffle=False)
    calculate_combined_accuracy(args, raw_test_loader, sigma=args.sigma)

    print("Corrupted testset:")
    corr_test_loader = torch.utils.data.DataLoader(corr_test_dataset,
    batch_size= args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,corr_test_loader, sigma=args.sigma)

    print("Reconstructed testset:")
    recon_test_loader = torch.utils.data.DataLoader(recon_test_dataset,
    batch_size=args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,recon_test_loader, sigma=args.sigma)


    print("Reconstructed fill testset:")
    recon_fill_test_loader = torch.utils.data.DataLoader(recon_fill_test_dataset,
    batch_size=args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,recon_fill_test_loader, sigma=args.sigma)


    print("Mean Fill testset")
    mean_fill_test_loader = torch.utils.data.DataLoader(mean_fill_test_dataset,
    batch_size=args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,mean_fill_test_loader, sigma=args.sigma)

    print("Linear Interpolation testset:")
    linear_interp_test_loader = torch.utils.data.DataLoader(linear_interp_test_dataset,
    batch_size=args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,linear_interp_test_loader, sigma=args.sigma)


    evaluate_rmse(corr_test_x, recon_test,recon_fill_test, mean_fill_test, linear_interp_test,test_x)

    plot(test_x, corr_test_x, recon_test,recon_fill_test, mean_fill_test, linear_interp_test)



 ..from file ../../../../../data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
loading params...
torch.Size([9856, 1, 113, 24])
torch.Size([9856, 1, 113, 24])
torch.Size([9856, 1, 113, 24])
Raw testset:
Test Accuracy: 89.21469155844156 %
[[79.   4.9 13.8  1.4  0.8]
 [ 0.8 93.8  4.3  1.1  0. ]
 [ 2.9 14.3 82.4  0.4  0. ]
 [ 0.6  0.6  0.  98.6  0.1]
 [ 1.5  0.   0.2  3.  95.2]]
F1 score: 0.8912435729751317

Corrupted testset:
Test Accuracy: 67.58319805194805 %
[[67.9  1.9 28.3  1.2  0.6]
 [ 2.5 43.9 51.4  2.2  0. ]
 [ 5.8  5.7 88.1  0.4  0. ]
 [11.1  0.8 12.  76.1  0. ]
 [ 8.4  0.2  2.8  1.9 86.6]]
F1 score: 0.6810692389363461

Reconstructed testset:
Test Accuracy: 86.58685064935065 %
[[72.3  7.6 16.5  2.8  0.7]
 [ 0.9 95.7  2.   1.3  0. ]
 [ 2.2 23.5 73.8  0.5  0. ]
 [ 0.4  0.9  0.  98.5  0.1]
